In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, 'hello')
    await say_after(2, 'world')
    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [67]:
df = pd.read_csv('dataset_binary_games.csv')
df

,Unnamed: 0,id,track_name,price,cont_rating,prime_genre,vpp_lic,app_desc,is_games
0,0,302584613,"Kindle – Read eBooks, Magazines & Textbooks",0.00,4,book,1,turn kindle free kindle carry wherever go narr...,0
1,1,350480010,eBook: War and Peace,3.99,9,book,1,enjoy reading war peace day night multiple fon...,0
2,2,366869252,OverDrive – Library eBooks and Audiobooks,0.00,4,book,1,borrow streaming video library overdrive touch...,0
3,3,369111608,iBunkoHD,6.99,17,book,1,best way read comic realistic interface enjoy ...,0
4,4,379693831,"Audible – audio books, original series & podcasts",0.00,4,book,1,welcome audible company home unmatched selecti...,0
...,...,...,...,...,...,...,...,...,...
6235,6235,924373886,Crossy Road - Endless Arcade Hopper,0.00,9,games,1,apple design award winner chicken cross pigeon...,1
6236,6236,608899141,Cut the Rope: Time Travel,0.99,4,games,1,join om back time feed candy cut rope time tra...,1
6237,6237,1070525362,Cosmic Challenge: Best online space racing game,0.00,4,games,1,space race competition space race different sp...,1
6238,6238,1100810556,Hidden Expedition: Dawn of Prosperity - A Myst...,6.99,9,games,1,pay play forever unusual signal vicinity old a...,1


In [68]:
x_df = df.iloc[:, [3,4,6,7]]
y_df = df.iloc[:, [8]]

In [69]:
def tf_idf(text, max_feat=None):
    tfidf_vect = TfidfVectorizer(max_features=max_feat)
    tfidf_vect.fit(text)
    return tfidf_vect

In [70]:
base_vectorizer = tf_idf(df.app_desc, max_feat=600)

In [71]:
kf = KFold(n_splits=2)

In [108]:
def build_mn_naive_bayes(train, test, y_train, y_test, vectorizer):
    train_x_tfidf = vectorizer.fit_transform(train.app_desc).toarray()
    test_x_tfidf = vectorizer.fit_transform(test.app_desc).toarray()
#     print(train.iloc[:,0:3].values.tolist())
#     print(train_x_tfidf)
#     x_train = np.array([train.iloc[:,0:3].values.tolist(), train_x_tfidf]).reshape(-1,1)
#     x_test = [test.iloc[:,0:3], test_x_tfidf]
    mnb = MultinomialNB()
    mnb.fit(train_x_tfidf, y_train.values)
    print(y_train.values)
    predictions_mnb = mnb.predict(test_x_tfidf)
    print("mnb Accuracy Score -> ",accuracy_score(predictions_mnb, y_test.values)*100)
    return mnb

In [109]:
for train_index, test_index in kf.split(df):
    mn_naive_bayes = build_mn_naive_bayes(
        train = x_df.iloc[train_index,:], 
        test = x_df.iloc[test_index,:], 
        y_train = y_df.iloc[train_index,:],
        y_test = y_df.iloc[test_index,:],
        vectorizer = base_vectorizer
    )

C:\Users\muzaki\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
mnb Accuracy Score ->  0.0
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
mnb Accuracy Score ->  0.0


C:\Users\muzaki\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
x_df.iloc[:,0:3]